This demonstrate text generation using a character-based RNN. 

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words but consider

1. the model is character-based. When training started, the model did not know how to spell an English word, or that word were even a unit of text.

2. The structure of the output resemble a play - blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

3. as demonstrated, the model is trained on small batches of text (100 character each), and is still able to generate a longer sequence of text with coherent structure.

In [1]:
import tensorflow as tf 
import numpy as np 
import os  
import time

2023-01-24 22:06:34.397068: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 22:06:34.988514: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-24 22:06:38.084485: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/x86_64-linux-gnu/gazebo-11/plugins:/opt/ros/humble/opt/rviz_ogre_vendor/lib:/o

# Download the data

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt',
                'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 2s 1us/step


# Read the data

In [3]:
# read and decode for py2 compat
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# length of text is the number of characters in it 
print(f'length of text: {len(text)} characters')

length of text: 1115394 characters


In [4]:
# take a look at the first 250 characters in the text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# the unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


# Process the text 

Convert the string to a numerical representation

The tf.keras.layers.StringLookup layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [7]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [8]:
# now create tf.keras.layers.StringLookup layer 
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None
)

In [9]:
# it converts from token to character IDs
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Since the goal is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use 

tf.keras.layers.StringLookup(..., invert=True)

In [10]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None
)

This layer recovers the characters from the vectors of IDs, and return them as a tf.RaggedTensor of characters

In [11]:
chars = chars_from_ids(ids)

chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Can use tf.strings.reduce_join to join the characters back into strings

In [12]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [13]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

# The prediction task 

Given a character, or a sequence of characters, what is the most probable next character. This is the task the training model is gonna perform. The input to the model will be a sequence of characters, and you train the model to predict the output - the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character.

# Create training examples and targets

Divide the text into example sequences. Each input sequence will contain seq_length character from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length + 1. For example, say seq_length is 4 and our text is hello. The input sequence will be hell and the target sequence 'ello'

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices

In [14]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [15]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [16]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


The batch method let you convert the individual character to sequences of the desired size

In [18]:
seq_length = 100 

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


It is easier to see what this is doing if join the token back into strings

In [19]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


for training you will need a dataset of `(input, label)` pairs. Where `input` and `label` are sequences. At each time step the input is the current character and the label is the next character.

Here is a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep

In [20]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [21]:
# show the xample
split_input_target(list('Tensorflow'))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [22]:
dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
    print('Input: ', text_from_ids(input_example).numpy())
    print('Target: ', text_from_ids(target_example).numpy())

Input:  b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target:  b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


# Creating training batches 

Use `tf.data` to split text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches

In [23]:
# batch size
BATCH_SIZE = 64 

# buffer size to shuffle the dataset
# (tf data is designed to work with possibly infinite sequences, 
# so it does not attempt to shuffle the entire sequence in memory 
# . Instead, it maintains a buffer in which it shuffles elements)
BUFFER_SIZE = 10000 

dataset = (
    dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).\
            prefetch(tf.data.experimental.AUTOTUNE)
)

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

# Build the model

the model has three layers:

1. `tf.keras.layers.Embedding`: the input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions

2. `tf.keras.layers.GRU`: a type of RNN with size `units=rnn_units`. Also can use an LSTM layer here

3. `tf.keras.layers.Dense`: the output layer with `vocab_size` output. It output one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [24]:
# length of the vocabulary in StringLookup layer
vocab_size = len(ids_from_chars.get_vocabulary())

# the embedding dimension
embedding_dim = 256 

# number of rnn units
rnn_units = 1024 

In [25]:
class MyModel(tf.keras.Model):

    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs 
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x) 
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states 
        else:
            return x

In [26]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as the input, and applies the dense layer to generate logits predicting the log-likelihood of the next character.

# Try the model

run the model to see that it behaves as expected

In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


2023-01-24 22:31:10.005219: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401


(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [29]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


# Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state and the input this time step, preduct the class of the next character

# Attach an optimizer and loss function

In [30]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [31]:
model.compile(optimizer='adam', loss=loss)

# Configure checkpoint

In [32]:
# directory where the checkpoint will be saved
checkpoint_dir = './training_checkpoints'

# name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

# Execute the training

to keep training time reasonable, use 10 epochs to train the model.

In [33]:
EPOCHS = 20 

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20


2023-01-24 22:31:46.402987: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f0f0c017600 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-01-24 22:31:46.403091: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce MX350, Compute Capability 6.1
2023-01-24 22:31:46.479479: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-01-24 22:31:47.644764: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-01-24 22:31:47.744101: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


172/172 [==============================] - 82s 443ms/step - loss: 2.7086
Epoch 2/20
172/172 [==============================] - 79s 439ms/step - loss: 1.9848
Epoch 3/20
172/172 [==============================] - 83s 458ms/step - loss: 1.7091
Epoch 4/20
172/172 [==============================] - 83s 420ms/step - loss: 1.5494
Epoch 5/20
172/172 [==============================] - 78s 412ms/step - loss: 1.4516
Epoch 6/20
172/172 [==============================] - 83s 438ms/step - loss: 1.3847
Epoch 7/20
172/172 [==============================] - 82s 422ms/step - loss: 1.3314
Epoch 8/20


2023-01-24 22:41:23.300378: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 7171 of 10000
2023-01-24 22:41:24.813708: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


172/172 [==============================] - 85s 428ms/step - loss: 1.2876
Epoch 9/20


...

172/172 [==============================] - 86s 425ms/step - loss: 1.0841
Epoch 14/20
172/172 [==============================] - 84s 436ms/step - loss: 1.0386
Epoch 15/20
172/172 [==============================] - 84s 442ms/step - loss: 0.9912
Epoch 16/20
172/172 [==============================] - 84s 438ms/step - loss: 0.9406
Epoch 17/20
172/172 [==============================] - 83s 436ms/step - loss: 0.8896
Epoch 18/20
172/172 [==============================] - 91s 482ms/step - loss: 0.8374
Epoch 19/20


2023-01-24 22:57:08.519953: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 4964 of 10000
2023-01-24 22:57:11.085161: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


172/172 [==============================] - 85s 422ms/step - loss: 0.7855
Epoch 20/20


2023-01-24 22:58:34.072783: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 6766 of 10000
2023-01-24 22:58:36.152986: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


172/172 [==============================] - 88s 442ms/step - loss: 0.7371


# Generate text 

the simplest way to generate text with this model is to run it in a loop, and keep tract the models internal state as execute it

each time call the model, pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text

In [34]:
class OneStep(tf.keras.Model):

    def __init__(self, model, chars_from_ids, ids_from_char, temperature=1.0):
        super().__init__()
        self.temperature = temperature 
        self.model = model 
        self.chars_from_ids = chars_from_ids 
        self.ids_from_chars = ids_from_char 
    
        # create a mask to prevent '[UNK]' from being generated
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # put a -inf at each bad index 
            values = [-float('inf')] * len(skip_ids),
            indices = skip_ids,
            # match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())]
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function 
    def generate_one_step(self, inputs, states=None):
        # convert string to token id 
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor() 

        # run the model
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(inputs=input_ids, states=states, return_state=True)

        # only use the last prediction 
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits / self.temperature 
        # apply the prediction mask: prevent '[UNK]' from being generated
        predicted_logits = predicted_logits + self.prediction_mask 

        # sample the output logits to generate token ids 
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # convert from token ids to characters 
        predicted_chars = self.chars_from_ids(predicted_ids)

        # return the characters and model state
        return predicted_chars, states

In [35]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

run it in a loop to generate some text. Looking at the generated text, you will see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epoch, it has not yet learned to form coherent sentences.

In [36]:
start = time.time()
states = None 
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Tranious lord, I spake, but with usuted!
Thou dost a wife what an ass unto my father have I done;
Hold, together, wife, Signior Petrapial.

DUKE VINCENTIO:
Call it not patience.

SLY:
Ay, by my mother, she be as mistress
For heaven and my test thy mains a hoar?

KING HENRY VI:
Well mean, that, my mind to be a sea-maid,
For they must be and in the eager of itself.
Say we will perish.

CORIOLANUS:
I would, I come, inform them, nay, by remedy.

JULIET:
And to stand all achieve the tomb;
And being by your honour, look perfect,
Pray you, ay, as thy glories shall be framed.
Love not thy overthrow not? 'Zeauth it this into a tail
Once more, his army is young Gremio.

GREMIO:
Yea, yea, yea.

ISABELLA:
Some other widow or foreward thence, were here?

QUEEN ELIZABETH:
An offer of his judgment till he crown me
He was my wife't offine; and with the breath thou liest.

GLOUCESTER:
And I, a say to plead, but do not serve: it is my
son to be sif of both the only son of my blest,
And we would h

the easiest thing you can do to improve the results is to train it for longer (try epochs = 30)

# Export the generator 

This single step model can easily be saved and restored allowing you to use it anywhere a tf.saved_model is accepted

In [37]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets
